In [ ]:
#!/usr/bin/python
import json
import smtplib
import sys
import time
from email.mime.text import MIMEText
from pathlib import Path


def debounce(timeout=300):
    p = Path("EmailDebounceTracker.lock")
    if p.exists():
        if time.time() - p.stat().st_mtime > timeout:
            p.unlink()
            p.touch()
            return True
        else:
            return False
    else:
        p.touch()
        return True


p = Path("config.json")
if p.exists() == False:
    config = {
        "smtp_host": "",
        "from": "",
        "to": [""],
        "subject": "Automation - Service Down",
        "message": "The service has gone down",
        "Debounce_Seconds": 300,
    }
    p.write_text(json.dumps(config, indent=4))
    print("Config created, exiting")
    sys.exit(1)
else:
    config = json.loads(p.read_text())

try:
    msg = MIMEText(config["message"])
    msg["Subject"] = config["subject"]
    msg["From"] = config["from"]
    msg["To"] = ",".join(config["to"])
    smtpObj = smtplib.SMTP(config["smtp_host"])
    if debounce(config["Debounce_Seconds"]):
        smtpObj.sendmail(config["from"], config["to"], msg.as_string())
        print("Successfully sent email")
    else:
        print("debounce")
except Exception as e:
    print(e)